In [56]:
symbol = ["XLF","SPY","GDX","VXX","EEM"]

In [57]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pymongo
import time

## Using "Chay_db" Database 
client = pymongo.MongoClient()
db = client.Chay_db

In [58]:
for sym in symbol:
    url = "http://etfdb.com/etf/"+ sym
    html = urlopen(url)
    h_code = BeautifulSoup(html, 'html.parser')
    
    ### web scraping Vitals Section
    Info = h_code.findAll("ul", {"class": "list-unstyled"})
    Info_2 = h_code.findAll("div", {"id":"barchart-trading-data"})
    
    cont1 =Info[0].findAll("span",{"class":"pull-right"})
    cont2 =Info[0].findAll("span",{"class":None})
    
    vit_dict = {}
    for i in  list(range(8)):
        key = cont2[i-1].text.replace(":","")
        value = cont1[i-1].text
        vit_dict[key]=value
    
    ### web scraping Trading Data Section
    TD_dict = {}
    cont4 = Info_2[0].findAll("td")
    cont4=[i.text.replace(":","") for i in cont4]
    
    cont3=Info[1].findAll("span")
    cont3=[i.text.replace(":","") for i in cont3]
    
    cont5=Info[2].findAll("span")
    cont5=[i.text.replace(":","") for i in cont5]
    
    TD_dict = [{"Date":time.strftime("%x"),"Dat":{cont4[0]:cont4[1],cont4[2]:cont4[3],
                                                  cont4[4]:cont4[5],cont4[6]:cont4[7],
                                                  cont3[0]:cont3[1],cont3[2]:cont3[3],
                                                  cont5[0]:cont5[1],cont5[2]:cont5[3]}}]
    ### Insert into MongoDB
    data={"Symbol":sym,"Vitals":vit_dict,"Trading_Data":TD_dict}
    ### Put it in ETF_test Collection
    db.ETF_test.insert_one(data)


# To Update Trading Data

In [ ]:
client = pymongo.MongoClient()
db = client.Chay_db

In [59]:
for sym in symbol:
    url = "http://etfdb.com/etf/"+ sym
    html = urlopen(url)
    h_code = BeautifulSoup(html, 'html.parser')
    
    TD_dict = {}
    lastest = list(db.ETF_test.find({"Symbol":sym},{"Trading_Data.Date":1,"_id":0}))[0]["Trading_Data"]
    lastest = [i["Date"] for i in lastest]
    if time.strftime("%x") in lastest:
        print("The " + sym + " Trading Data Has been already updated")
    else:
        cont4 = Info_2[0].findAll("td")
        cont4=[i.text.replace(":","") for i in cont4]
    
        cont3=Info[1].findAll("span")
        cont3=[i.text.replace(":","") for i in cont3]
    
        cont5=Info[2].findAll("span")
        cont5=[i.text.replace(":","") for i in cont5]
    
        Lat_TDdict = [{"Date":time.strftime("%x"),"Dat":{cont4[0]:cont4[1],cont4[2]:cont4[3],
                                                             cont4[4]:cont4[5],cont4[6]:cont4[7],
                                                             cont3[0]:cont3[1],cont3[2]:cont3[3],
                                                             cont5[0]:cont5[1],cont5[2]:cont5[3]}}]
        db.ETF_test.update_one({"Symbol":sym},{"$push":{"Trading_Data":Lat_TDdict}})
        print("Finish Updating " + sym + "Trading Data")
    

The XLF Trading Data Has been already updated
The SPY Trading Data Has been already updated
The GDX Trading Data Has been already updated
The VXX Trading Data Has been already updated
The EEM Trading Data Has been already updated
